In [ ]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pickle
from copy import deepcopy
from matplotlib.lines import Line2D

### China Committed Emissions
The emissions committed by China by virtue of existing fossil based infastructure

In [ ]:
# committed emissions data for china
china_commit = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/committed.csv', index_col = 0)
china_commit = china_commit.drop(columns = ['all.emissions.energy.2018', 'all.emissions.energy.2019', 'all.emissions.energy.2020'])

# multiply data by 1000 to convert GT to MT
china_commit['country'] = china_commit['country'].astype(str)
china_commit.iloc[:, china_commit.columns != 'country'] = china_commit.iloc[:, china_commit.columns != 'country'].astype(float) * 1000

In [ ]:
# select china data
commit_emit_china = china_commit[china_commit['country'] == 'China']

# combine industry and energy emissions into one variable
new_year_names = []
for col_name in commit_emit_china.columns:
    new_year_names.append(col_name[-4:])
commit_emit_china.columns = new_year_names
commit_emit_china = commit_emit_china.groupby(commit_emit_china.columns, axis=1).sum()

# add prefix to col name of totals 
commit_emit_china = commit_emit_china.add_prefix('commit.')

# rename country column to use in later merge
commit_emit_china = commit_emit_china.rename(columns = {'commit.ntry':'country'})

### China Considered Emissions
The global considered emissions: projects planned and their potential future emissions. This data is my considered emissions (coal and gas plant) data processed by Johannes to produce totals across emission source.

- gas and coal data in MW of electrical generation capacity, not emissions units

In [ ]:
# read in considered gas and coal emissions csvs
gas_consid = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/gas_data/gas-considered.csv', index_col = 0)
coal_consid = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/coal_data/coal-considered.csv', index_col = 0) 
steel_consid = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/steel_data/steel_panel_clean.csv', index_col = 0)

# convert coal data from MT to GT CO2
gas_consid.iloc[:, gas_consid.columns != 'country'] = gas_consid.iloc[:, gas_consid.columns != 'country'].astype(float)
coal_consid.iloc[:, coal_consid.columns != 'country'] = coal_consid.iloc[:, coal_consid.columns != 'country'].astype(float)
steel_consid.iloc[:, steel_consid.columns != 'country'] = steel_consid.iloc[:, steel_consid.columns != 'country'].astype(float)

# # combine these by appending the dfs
gas_coal_consid = gas_consid.append(coal_consid, ignore_index = True)
gas_coal_steel_consid = gas_coal_consid.append(steel_consid, ignore_index = True)

In [ ]:
# combine gas and coal data into one variable
years_list = []
for i in gas_coal_steel_consid.columns:
    years_list.append(i[-4:])
gas_coal_steel_consid.columns = years_list   
coal_gas_col_totals = gas_coal_steel_consid.groupby(gas_coal_steel_consid.columns, axis=1).sum()

# add prefix to col name of totals 
considered_emit_totals = coal_gas_col_totals.add_prefix('gas+coal+steel.emissions.proposed.')

# rename cut off country column
considered_emit_totals_proper = considered_emit_totals.rename(columns = {'gas+coal+steel.emissions.proposed.ntry':'country'})

# put country column at the front of the df
col = considered_emit_totals_proper.pop('country')
considered_emit_totals_proper.insert(0, col.name, col)

In [ ]:
# select out China data
considered_emit_china1 = considered_emit_totals_proper[(considered_emit_totals_proper['country'] == 'China') ^ (considered_emit_totals_proper['country'] == 'EU27')]

Add a scaling factor to the considered emissions: by about what factor do these underestimate real global emissions?

In [ ]:
considered_emit_china2 = considered_emit_china1.iloc[0]
flt_vals = []

for i in considered_emit_china2.values[1:]:
    i = pd.to_numeric(i, downcast = 'float')
    
    #arbitrary multiplication factor for gas and coal emissions, to scale it up to being a proxy for all proposed emissions
    flt_vals.append(i * 2)

considered_emit_china2.values[1:] = flt_vals
considered_emit_china = pd.DataFrame(considered_emit_china2).transpose()

### Global Emissions Based on Scenario
This is the expected trajectory of emissions based on SSP and RCP scenario combinations

In [ ]:
scenario = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/emissions_intensity/region_CO2_by_scenario--PLANNED.EMIT.csv', index_col = 0)

### Data Combination

In [ ]:
# add region column to scenario data
considered_emit_china['region'] = 'China'

# select out post 2021 columns for scenario data
post2021_scen = pd.concat([scenario.iloc[:,0:3], scenario.iloc[:,174:].add_prefix('planned.')], axis = 1)
china_post2021_scen = post2021_scen[post2021_scen['region'] == 'China']

In [ ]:
# CCS stands for "considered, committed, & scenario"
CCS_chinaA = china_post2021_scen.merge(commit_emit_china, left_on = 'region', right_on = 'country').drop(columns = 'country')
CCS_chinaB = CCS_chinaA.merge(considered_emit_china, on = 'region')
CCS_china = CCS_chinaB.drop(columns = 'country', axis = 1)
CCS_china.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/CHINA_considered_committed_scenario.csv')

In [ ]:
#Disaggregated gas, coal, steel version for Johannes
# merge this with committed and scenario data

gcs = gas_consid.merge(coal_consid, on = 'country').merge(steel_consid, on= 'country')
gcs_china = gcs[gcs['country'] == 'China']
gcs_china = gcs_china.iloc[:, 1:] * 2
gcs_china['country'] = 'China'
commit_gcs = pd.merge(gcs_china, commit_emit_china, on= 'country', how = 'outer')

all_disagg = commit_gcs.merge(china_post2021_scen, left_on='country', right_on='region')
country = all_disagg.pop('country')
rcp = all_disagg.pop('SSP')
ssp = all_disagg.pop('RCP')

all_disagg.insert(0, 'Country', country)
all_disagg.insert(1, 'SSP', rcp)
all_disagg.insert(2, 'RCP', ssp)

all_disagg.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/all_vars_disagg.csv')

### Data Plots

In [ ]:
flipped = CCS_china.transpose()
data_per_scenario = {}

for column in flipped.columns:
    current_scenario = flipped[column][['region','SSP','RCP']]
    
    scenario_vals = []
    committed_vals = []
    considered_vals = []
    
    commit_add = []
    consid_add = []
    
    for var in flipped.index:
    
        if 'planned' in var:
            if flipped[column][var] != 0:
                if float(var[-4:]) >= 2021:
                    scenario_vals.append([var[-4:], flipped[column][var]])                
        elif 'commit' in var:
            commit_add.append(flipped[column][var])
            if flipped[column][var] != 0:
                if float(var[-4:]) >= 2021:
                    committed_vals.append([var[-4:], flipped[column][var]])
        elif 'proposed' in var:
            consid_add.append(flipped[column][var])
            if flipped[column][var] != 0:
                if float(var[-4:]) >= 2021:
                    considered_vals.append([var[-4:], flipped[column][var]])

    data_per_scenario[tuple(current_scenario)] = {'planned': np.array(scenario_vals, dtype = float), 
                                           'commit': np.array(committed_vals, dtype = float), 
                                           'considered_vals': np.array(considered_vals, dtype = float)}
    

In [ ]:
for i, scenario in enumerate(data_per_scenario):
    plt.figure(figsize=(10,3))

    for key in data_per_scenario[scenario]:
        plt.plot(data_per_scenario[scenario][key][:, 0],
                    data_per_scenario[scenario][key][:, 1].astype(float), label = str(key));

    plt.xlabel('Year')
    plt.ylabel('Emissions')
    plt.title('Planned, committed, and considered emissions till 2100 ' + str(scenario))
    plt.legend(loc = 'upper right', bbox_to_anchor=(1.2, 1))

    plt.savefig('/Users/Violet/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_plots/' + str(scenario) + '.jpeg')
    
    #i==27 runs all scenarios
    if i==27: 
        break

### New Variable Creation
- considered + committed 
- (considered + committed) / scenario
- committed / scenario

In [ ]:
d = deepcopy(data_per_scenario)
#d_expect = deepcopy(data_per_scenario)
all_years = np.arange(2021, 2100, dtype = int)

for scenario in d:
    data = d[scenario]
    
    committed_dict = {x[0]: x[1] for x in data['commit']}
    considered_dict = {x[0]: x[1] for x in data['considered_vals']}
    planned_dict = {x[0]: x[1] for x in data['planned']}
    
    
    data['sum_comm_considered'] = {}
    data['frac_(comm+consid)/scen'] = {}
    data['frac_comm/scen'] = {}
    data['comm+consid+scen'] = {}
    data['expectable'] = {}
    
    # sum of committed and considered
    for year in all_years:
        if year >= 2021:
            if year in committed_dict or year in considered_dict:
                data['sum_comm_considered'][year] = 0
            if year in committed_dict:
                data['sum_comm_considered'][year] += committed_dict[year]
            if year in considered_dict:
                data['sum_comm_considered'][year] += considered_dict[year]
    
    data['sum_comm_considered'] = np.array(list(data['sum_comm_considered'].items()))
    consid_commit_sum_dict = {x[0]: x[1] for x in data['sum_comm_considered']}
    
    # (considered + committed) / scenario
    for year in all_years:
        if year in consid_commit_sum_dict:
            data['frac_(comm+consid)/scen'][year] = 0
        if year not in consid_commit_sum_dict or year not in planned_dict:
            data['frac_(comm+consid)/scen'][year] = 0
        if year in planned_dict and year in consid_commit_sum_dict:
            data['frac_(comm+consid)/scen'][year] += (consid_commit_sum_dict[year] / planned_dict[year])

    data['frac_(comm+consid)/scen'] = np.array(list(data['frac_(comm+consid)/scen'].items()))
    
    # committed / scenario
    for year in all_years:
        if year in committed_dict or year in planned_dict:
            data['frac_comm/scen'][year] = 0
        if year not in committed_dict or year not in planned_dict:
            data['frac_comm/scen'][year] = 0
        if year in planned_dict and year in considered_dict:
            data['frac_comm/scen'][year] += (committed_dict[year] / planned_dict[year])

    data['frac_comm/scen'] = np.array(list(data['frac_comm/scen'].items()))
    
    # considered + committed + scenario
    for year in all_years:
        if year in planned_dict:
            data['comm+consid+scen'][year] = planned_dict[year]
        if year in planned_dict and year in consid_commit_sum_dict:
            data['comm+consid+scen'][year] += (consid_commit_sum_dict[year])

    data['comm+consid+scen'] = np.array(list(data['comm+consid+scen'].items()))
    
    # planned - (considered + committed)
    for year in all_years:
        if year in planned_dict or year in consid_commit_sum_dict:
            data['expectable'][year] = 0
        if year not in planned_dict or year not in consid_commit_sum_dict:
            data['expectable'][year] = 0
        if year in planned_dict and year in consid_commit_sum_dict:
            data['expectable'][year] += (planned_dict[year] - consid_commit_sum_dict[year])
            
    data['expectable'] = np.array(list(data['expectable'].items()))
    
    d[scenario] = deepcopy(data)
    #d_expect[scenario] = deepcopy(data)

### Export data to DF

In [ ]:
def make_colnames(d):
    some_scenario = list(d.keys())[0]
    col_names = ['region', 'SSP', 'RCP']
    
    for emission_var in d[some_scenario]:
        years = d[some_scenario][emission_var][:, 0]
        col_names += [emission_var + '.' + str(int(year)) for year in years]
    return col_names

def data_to_R_format(d):
    colnames = make_colnames(d)    
    cols = {col: [] for col in colnames}
    for scenario in d:
        cols['region'].append(scenario[0])
        cols['SSP'].append(scenario[1])
        cols['RCP'].append(scenario[2])
        
        for emission_var in d[scenario]:
            for year, val in d[scenario][emission_var]:
                cols[emission_var + '.' + str(int(year))].append(val)
        
    return cols
    
data_dict = data_to_R_format(d)
ccs_all_vars = pd.DataFrame.from_dict(data_dict, orient = 'columns')
ccs_all_vars.to_csv('/Users/Violet/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/CCS_ALL_VARS.csv')

#### Create a version of the data which has the necessary variables for plotting affectable emissions
- see the affectability.ipynb script for the affectable emissions implementation

In [ ]:
# Removing the following columns from the data: 
    #frac_commit/scen
    #commit+consid+scen
    #frac_(commit+consid)/scen
    #sum_committed_considered
    #considered_vals
    
# Keeping the following columns
    #committed, planned, expected but uncommitted 
# Adding back the disaggragated data (coal, gas, and steel) to replace the excluded considered emissions

commit_planned_expect_cols = [c for c in ccs_all_vars.columns if 'planned' in c or 'region' in c or 'expectable' in c or 'SSP' in c or 'RCP' in c or 'commit' in c]
ccs_filtered = ccs_all_vars[commit_planned_expect_cols]

consid_disagg_cols = [c for c in all_disagg.columns if 'gas' in c or 'coal' in c or 'steel' in c or 'SSP' in c or 'RCP' in c or 'Country' in c]
disagg_filtered = all_disagg[consid_disagg_cols]

disagg_all_vars = ccs_filtered.merge(disagg_filtered, left_on = ['SSP', 'RCP', 'region'], right_on = ['SSP', 'RCP', 'Country'])
disagg_all_vars.to_csv('/Users/Violet/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/all_vars_w_disagg_considered_vals.csv')

### Stacked Plots

In [ ]:
flipped_stack = ccs_all_vars.transpose()
data_stack = {}

for column in flipped.columns:
    current_scenario = flipped_stack[column][['region','SSP','RCP']]
    
    sum_committed_considered = []
    committed_vals = []
    planned_vals = []
    commit_consid_scen_sum = []
    
    for var in flipped_stack.index:
        if 'sum_comm_considered' in var:
            sum_committed_considered.append([var[-4:], flipped_stack[column][var]])                
        elif 'commit' in var:
            committed_vals.append([var[-4:], flipped_stack[column][var]])
        elif 'planned' in var:
            planned_vals.append([var[-4:], flipped_stack[column][var]])
        elif 'comm+consid+scen' in var:
            commit_consid_scen_sum.append([var[-4:], flipped_stack[column][var]])

    data_stack[tuple(current_scenario)] = {'Expected Emissions ' + str(current_scenario.values[0]) + ' (' + str(current_scenario.values[1]) + str(', RCP' + current_scenario.values[2]) + ')': np.array(planned_vals, dtype = float),
                                           'Committed Emissions': np.array(committed_vals, dtype = float), 
                                           'Considered Emissions': np.array(sum_committed_considered, dtype = float),
                                           'Total (consid, commit, scen)': np.array(commit_consid_scen_sum, dtype = float)
                                           }

In [ ]:
custom_lines = [Line2D([0], [0], color='darksalmon', lw=2), 
                Line2D([0], [0], color='sandybrown', lw=2), 
                Line2D([0], [0], color='cadetblue', lw=2),
                Line2D([0], [0], color='black', linestyle = '--', lw=2)]

for i, scenario in enumerate(data_stack):
    plt.figure(figsize=(10,3.2))

    for key in data_stack[scenario]:
        if 'Expected Emissions' in key:
            plt.plot(data_stack[scenario][key][:, 0], data_stack[scenario][key][:, 1].astype(float), label = str(key))#, alpha = 0);
            plt.fill_between(data_stack[scenario][key][:, 0], data_stack[scenario][key][:, 1], color = 'cadetblue', capstyle = 'round')#, alpha = 0.8)
            plt.fill_between(data_stack[scenario]['Considered Emissions'][:, 0], data_stack[scenario]['Considered Emissions'][:, 1], data_stack[scenario]['Committed Emissions'][:, 1], color = 'darksalmon', capstyle = 'round', linestyle = 'dotted')#, alpha = 0.8)
            plt.fill_between(data_stack[scenario]['Committed Emissions'][:, 0], data_stack[scenario]['Committed Emissions'][:, 1], color = 'sandybrown')#, alpha = 0.8)
            plt.legend(custom_lines, ['Considered Emissions','Committed Emissions', key], loc = 'upper right', bbox_to_anchor=(1, 1))

        else:
            plt.plot(data_stack[scenario][key][:, 0],
                    data_stack[scenario][key][:, 1].astype(float), label = str(key), alpha = 0);
            
    plt.hlines(0, 2021, 2100, color = 'black', linestyle = '--', label = 'Negative Emissions')    
    plt.xlim(2021,2100)
    
    plt.xlabel('Year')
    plt.ylabel('Emissions (MtCO2/yr)')
    plt.title('Expected, committed, and considered emissions till 2100 ' + str(scenario))
    plt.savefig('/Users/Violet/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_plots/' + 'stacked' + str(scenario) + '.jpeg', transparent = True, bbox_inches = 'tight', dpi = 1000)
    
    #('China', 'SSP3', '1.9') and ('China', 'SSP3', '2.6') not included in the affectability data, so:
    #i==25 runs all scenarios
    if i==25: 
        break